<a href="https://colab.research.google.com/github/abhinavnautiyalDS/Book-recommender-system/blob/main/Book_recommeder_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


#dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
Rating=pd.read_csv("/content/Ratings (1).csv")
users=pd.read_csv('/content/Users (1).csv')
book=pd.read_csv("/content/Books (1).csv")

In [ ]:
book.shape

In [ ]:
Rating.shape

In [ ]:
users.shape

###**Book dataset**

In [ ]:
len(book['Book-Title'].unique())

In [ ]:
print("No of columns ",book.shape[1]," and rows ",book.shape[0])

In [ ]:
#checking null values
book.isnull().sum()

In [ ]:
#checking duplicate values
print("duplicate values ",book.duplicated().sum())

In [ ]:
#info
book.info()

In [ ]:
#describe
book.describe()

### **Rating dataset**

In [ ]:
Rating.head()

In [ ]:
Rating.isna().sum()

In [ ]:
print("No of rows ",Rating.shape[0],"No of columns ",Rating.shape[1])

In [ ]:
#duplicate values
print("duplicate values ",Rating.duplicated().sum())

In [ ]:
#info
Rating.info()

In [ ]:
#describe

Rating.describe()

###**Users dataset**

In [ ]:
users.head()

In [ ]:
#null values
users.isna().sum()

In [ ]:
#duplicate values
print("duplicate rows",users.duplicated().sum())

In [ ]:
#shape
print("No of rows ",users.shape[0],"no of columns ",users.shape[1])

In [ ]:
#info
users.info()

In [ ]:
#describe
users.describe()

#  **Merging**

In [ ]:
book.shape

In [ ]:
Rating.shape

In [ ]:
users.shape

In [ ]:
#since  ISBN is common between book and Rating dataset and user_id is common in Rating and users we will gonna merge them

In [ ]:
merge_df=book.merge(Rating,on='ISBN',how='inner')

In [ ]:
merge_df.head()

In [ ]:
merge_df=merge_df.merge(users,on='User-ID',how='inner')

In [ ]:
len(merge_df['Book-Title'].unique())

In [ ]:
len(merge_df['User-ID'].unique())

In [ ]:
merge_df.shape

#**Mege_df**

In [ ]:
merge_df.head()

In [ ]:
#null values
merge_df.isna().sum()

##### **Handling null and outlier value in Age column**

In [ ]:
print("Age columnn cotain ",round(merge_df['Age'].isna().sum()*100/merge_df.shape[0],2),"% null values")

In [ ]:
x=merge_df.Age.plot(kind='box')

In [ ]:
#Handling outlier
merge_df.Age.describe()

In [ ]:
merge_df["Age"].quantile(0.99)


**Minimum age is zero and maximum age is 244 which is not possible , so i am replacing value less then 10 with 5 percentile value and value greater than 69 with 98 percentile value**


In [ ]:
merge_df.shape

In [ ]:
import numpy as np

lower_bound = merge_df["Age"].quantile(0.05)  # 5th percentile
upper_bound = merge_df["Age"].quantile(0.98)  # 98th percentile

merge_df["Age2"] = np.where(merge_df["Age"] < 10, lower_bound,
                            np.where(merge_df["Age"] > 69, upper_bound, merge_df["Age"]))


In [ ]:
merge_df

In [ ]:
#Handling outliers
#replace null values with mean value

merge_df.Age2=merge_df.Age2.fillna(merge_df.Age2.mean())


In [ ]:
merge_df.Age2=merge_df.Age2.astype('int')

In [ ]:
merge_df.Age.plot(kind='box')

In [ ]:
merge_df.Age2.plot(kind='box')

In [ ]:
merge_df=merge_df.dropna()
merge_df.isna().sum()

## **Checking bookRating**

In [ ]:
merge_df['Book-Rating'].plot(kind='box')

In [ ]:
#no outlier no missing value

###**Checking unique values**

In [ ]:
merge_df['Book-Title'].unique()

In [ ]:
merge_df['Book-Author'].unique()

In [ ]:
merge_df['Year-Of-Publication']=merge_df['Year-Of-Publication'].astype('int') #this was in object format convert it in string

In [ ]:
merge_df['Year-Of-Publication'].describe()

In [ ]:
#2050 mein kan si book publish hui hai its is not possible
merge_df = merge_df[(merge_df['Year-Of-Publication'] <= 2025) &
                     (merge_df['Year-Of-Publication'] >= merge_df["Year-Of-Publication"].quantile(0.02))]


In [ ]:
#i have remove books which were publish after 2025 and before 1968 , these rows were 29

In [ ]:
merge_df['Year-Of-Publication'].describe()

In [ ]:
#

**publisher**

In [ ]:
merge_df['Publisher'].unique()


**User-ID**

In [ ]:
merge_df['User-ID'].unique()

**Location**

In [ ]:
merge_df['Location'].unique()

In [ ]:
len(merge_df['Book-Title'].unique())

### **checking duplicate columns**

In [ ]:
len(merge_df['User-ID'].unique())

In [ ]:
merge_df["Image-URL-M"].equals(merge_df["Image-URL-L"])


In [ ]:
merge_df["Image-URL-M"].equals(merge_df["Image-URL-S"])


In [ ]:
merge_df["Image-URL-L"].equals(merge_df["Image-URL-S"])


#**Exploratory Data Analysis (EDA)**

In [ ]:
merge_df.columns

###**Distrubution of numerical features**

In [ ]:

def visualise_numerical(df):

    numerical_features = ['Year-Of-Publication', 'User-ID', 'Book-Rating', 'Age2']

    # Filter only numerical features present in the DataFrame
    numerical_features = [col for col in numerical_features if col in merge_df.columns]

    if not numerical_features:
        print("No numerical features found in the dataset.")
        return

    # Set figure size
    plt.figure(figsize=(15, 10))

    # Plot each numerical feature
    for i, feature in enumerate(numerical_features, 1):
        plt.subplot(2, 3, i)  # Create subplots
        sns.histplot(merge_df[feature].dropna(), bins=30, kde=True, color="skyblue")
        plt.title(f'Histogram of {feature}')
        plt.xlabel(feature)
        plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()


In [ ]:
print(visualise_numerical(merge_df))

In [ ]:
# most of the books are published in the early 2000's
# most of the books have zero rating
# most of readers are of age 30-40

###**Top 20 Most popular books by number of votes**


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_book_title_counts(df, top_n=20):

    plt.figure(figsize=(12, 6))

    # Get top N most common books
    book_counts = df['Book-Title'].value_counts().nlargest(top_n)

    sns.barplot(
        x=book_counts.index,
        y=book_counts.values,
        palette="coolwarm"
    )

    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate for readability
    plt.ylabel("Count", fontsize=12)
    plt.title(f"Top {top_n} Most Common Book Titles", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.show()

plot_book_title_counts(merge_df, top_n=20)

### **Top 20 Most popular authors by number of votes**

In [ ]:
merge_df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_book_title_counts(df, top_n=20):

    plt.figure(figsize=(12, 6))

    # Get top N most common books
    book_counts = df['Book-Author'].value_counts().nlargest(top_n)

    sns.barplot(
        x=book_counts.index,
        y=book_counts.values,
        palette="coolwarm"
    )

    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate for readability
    plt.ylabel("Count", fontsize=12)
    plt.title(f"Top {top_n} Most popular Authors", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.show()

plot_book_title_counts(merge_df, top_n=20)

### **Most popular publisher**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_book_title_counts(df, top_n=20):

    plt.figure(figsize=(12, 6))

    # Get top N most common books
    book_counts = df['Publisher'].value_counts().nlargest(top_n)

    sns.barplot(
        x=book_counts.index,
        y=book_counts.values,
        palette="coolwarm"
    )

    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate for readability
    plt.ylabel("Count", fontsize=12)
    plt.title(f"Top {top_n} Most popular Publishers", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.show()

plot_book_title_counts(merge_df, top_n=20)

In [ ]:
merge_df.columns

### **The best book reader**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_book_title_counts(df, top_n=20):

    plt.figure(figsize=(12, 6))

    # Get top N most common books
    book_counts = df['User-ID'].value_counts().sort_values(ascending=False).nlargest(top_n)

    sns.barplot(
        x=book_counts.index,
        y=book_counts.values,
        palette="coolwarm"
    )

    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate for readability
    plt.ylabel("Count", fontsize=12)
    plt.title(f"Top {top_n} Best book enthusiast", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.show()

plot_book_title_counts(merge_df, top_n=20)

In [ ]:
merge_df.columns

Best location for readers

## **feature engneering**

**location conatin , city , state and contry information so i am gonna split theses**

In [ ]:
merge_df['City']=merge_df['Location'].str.replace(r'^\d+\s*', '', regex=True).str.split(',').str[0].str.strip()


In [ ]:
merge_df['State']=merge_df['Location'].str.replace(r'^\d+\s*', '', regex=True).str.split(',').str[1].str.strip()


In [ ]:
merge_df['Country']=merge_df['Location'].str.replace(r'^\d+\s*', '', regex=True).str.split(',').str[2].str.strip()

In [ ]:
df=merge_df.copy()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_book_title_counts(df, top_n=20):

    plt.figure(figsize=(12, 6))

    # Get top N most common books
    book_counts = df['Country'].value_counts().sort_values(ascending=False).nlargest(top_n)

    sns.barplot(
        x=book_counts.index,
        y=book_counts.values,
        palette="coolwarm"
    )

    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate for readability
    plt.ylabel("Count", fontsize=12)
    plt.title(f"Top {top_n} Reader's country", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.show()

plot_book_title_counts(merge_df, top_n=20)

### **Most popular state of book lovers in USA**

In [ ]:
merge_df['Country'].unique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_book_title_counts(df, top_n=20):

    plt.figure(figsize=(12, 6))

    # Get top N most common books
    book_counts = df[df['Country']=='usa']['State'].value_counts().sort_values(ascending=False).nlargest(top_n)

    sns.barplot(
        x=book_counts.index,
        y=book_counts.values,
        palette="coolwarm"
    )

    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate for readability
    plt.ylabel("Count", fontsize=12)
    plt.title(f"Top {top_n} Reader's State in USA", fontsize=14)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.show()

plot_book_title_counts(merge_df, top_n=20)

In [ ]:
merge_df.columns

### **Bivariant analysis**

In [ ]:
#Which publishers have the highest-rated books?
# Get publishers with more than 200 books
popular_publishers = merge_df['Publisher'].value_counts()
popular_publishers = popular_publishers[popular_publishers > 200].index  # Get only the publisher names

# Filter dataset for these publishers and compute average rating
publisher_ratings = (
    merge_df[merge_df['Publisher'].isin(popular_publishers)]
    .groupby('Publisher')['Book-Rating']
    .mean()
    .sort_values(ascending=False).head(20).reset_index()
)
sns.barplot(data=publisher_ratings,x='Publisher',y='Book-Rating')
plt.xticks(rotation=45, ha="right", fontsize=10)
#merge_df.groupby('Publisher')['Book-Rating'].mean().sort_values(ascending=False)

### **Model building**

We are building a recommendor system on the basis of how much user have rated a book. and now we'll consider only those users who are exprerienced readers who have reviewed atleasr 200 books and those books having minimun 50 reviews.

In [ ]:
merge_df1=merge_df.copy()

In [ ]:
#filtering 1st , experienced readers

filter1=merge_df['User-ID'].value_counts().reset_index()
filter1=filter1[filter1['count']>50]

merge_df=merge_df[merge_df['User-ID'].isin(filter1['User-ID'])]


In [ ]:
merge_df.shape

In [ ]:
merge_df

In [ ]:
#filter2

filter2=merge_df['Book-Title'].value_counts().reset_index()
filter2=filter2[filter2['count']>50]

filter2

In [ ]:
merge_df=merge_df[merge_df['Book-Title'].isin(filter2['Book-Title'])]

In [ ]:
merge_df.shape

In [ ]:
#checking duplicates
merge_df['User-ID'].unique()

In [ ]:
data=merge_df.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [ ]:
#filling null values with 0
data.fillna(0,inplace=True)

In [ ]:
data

In [ ]:
# now what we'll gonna do

#each book is represented nth dimestionvector

#so we'll cacluclate ecucleadean distance between each vector and samll desitance vectors would be similiar

#for this we will calucalte cosine similarity of each book with all books and those books having higher similiar scores are considerd
#similiar book of our target one

In [ ]:
#cosine similarty
# Cosine similarity is a metric used to measure how similar two vectors are, regardless of their magnitude. It is commonly used in text analysis, recommendation systems, and clustering problems.

# Formula
# The cosine similarity between two vectors  A and B = A.B/|A||B|



# Range of Cosine Similarity

# +1 → The vectors are identical (pointing in the same direction).


# 0 → The vectors are orthogonal (no similarity).


# −1 → The vectors are opposite (pointing in opposite directions).

# Use Cases
# Text Similarity: Represent documents as word frequency or TF-IDF vectors and compare their similarity.

# Recommendation Systems: Find similar users or items based on user-item interaction vectors.

# Clustering & Classification: Used in K-means clustering and nearest neighbor-based algorithms.

**Cosine similarity**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Compute cosine similarity
similarity = cosine_similarity(data)



In [ ]:
similarity

In [ ]:
similarity.shape

In [ ]:
sorted_similarities = sorted([(index, float(score)) for index, score in list(enumerate(similarity[0]))],key=lambda x: x[1], reverse=True)[1:10]



In [ ]:
sorted_similarities

In [ ]:
print(np.where(data.index=='1984')[0][0])

In [ ]:
df.columns

In [ ]:
data.shape

In [ ]:
import numpy as np
import pandas as pd

def recommend(book_name, data, similarity, merge_df1):
    # Normalize book name for matching
    book_name = book_name.strip().lower()

    # Normalize data index
    data.index = data.index.str.strip().str.lower()

    # Check if book exists in index
    if book_name not in data.index:
        print(f"'{book_name}' not found in dataset index.")
        return []

    # Fetch index of the book
    index = np.where(data.index == book_name)[0][0]

    # Ensure similarity matrix shape matches data
    if similarity.shape[0] != len(data):
        print("Warning: Similarity matrix size doesn't match dataset!")
        return []

    # Get top 5 similar books using argsort (excluding itself)
    similar_indices = np.argsort(-similarity[index])[1:6]  # Negative for descending sort

    # Get book titles safely
    book_titles = data.iloc[similar_indices].index.tolist()

    # Debugging logs
    print(f"Recommendations for '{book_name}': {book_titles}")

    # Normalize book titles for matching
    merge_df1["Book-Title"] = merge_df1["Book-Title"].str.strip().str.lower()
    book_titles = [title.lower() for title in book_titles]

    # Filter books in DataFrame
    info = merge_df1[merge_df1["Book-Title"].isin(book_titles)][["Book-Title", "Book-Author", "Year-Of-Publication", "ISBN"]]

    # Ensure no duplicates in recommendations
    return info.drop_duplicates(subset="Book-Title", keep="first").to_dict(orient="records")


In [ ]:
recommend('animal farm',data, similarity, merge_df1)

In [ ]:
book

In [ ]:
d=book.merge(Rating,on='ISBN')
d

In [ ]:
d['Book-Title'].value_counts().sort_values(ascending=False)

### **Saving model for deployment purpose**

In [ ]:
import pickle

pickle_filename = 'recommend.pkl'  # Save as a .pkl file

data_to_save = {
    "data": data,  # Ensure data is a DataFrame or dictionary
    "merge_df1": merge_df1,
    "similarity": similarity,  # If needed for the recommendation function
}

# Save the model and functions to a pickle file
with open(pickle_filename, 'wb') as file:#pickle.dump(model, open("RandomForestModel.pkl", "wb"))

    pickle.dump(data_to_save, file)

print(f"✅ Model saved to {pickle_filename}")
